In [10]:
import tensorflow as tf

x1 = tf.constant(5)
x2 = tf.constant(6)

result = tf.multiply(x1,x2)

sess = tf.Session()
print(sess.run(result))


    


30


In [11]:
import tensorflow as tf

'''
input > weight > hidden layer 1 (activation function)  > weights > hidden l 2 (activation function) > weights > 
output layer

compare output to intended output > cost function (cross entropy)
optimization function(optimizer) > minimize cost (AdamOptimizer....SGD, AdaGrad)

backpropogation

feed forward+backprop = epoch

'''


'\ninput > weight > hidden layer 1 (activation function)  > weights > hidden l 2 (activation function) > weights > \noutput layer\n\ncompare output to intended output > cost function (cross entropy)\noptimization function(optimizer) > minimize cost (AdamOptimizer....SGD, AdaGrad)\n\nbackpropogation\n\nfeed forward+backprop = epoch\n\n'

In [25]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

n_nodes_hl1 = 300
n_nodes_hl2 = 200
n_nodes_hl3 = 100

n_classes = 10
batch_size = 100

#height x width
x = tf.placeholder('float',[None, 784])
y = tf.placeholder('float')

def neural_network_model(data):
    

    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([784, n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}
    
    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}
    
    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}
    
    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                      'biases':tf.Variable(tf.random_normal([n_classes]))}
    
        # (input data * weights) + biases
        
    l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)
    
    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)
    
    l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)
    
    output = tf.matmul(l3, output_layer['weights']) + output_layer['biases']
    
    return output

def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(  tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    #      learning_rate_default = 0.001
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    #cycles feed forward + backprop
    hm_epochs = 20
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([optimizer, cost], feed_dict = {x: epoch_x, y: epoch_y})
                epoch_loss += c
            print('Epoch', epoch, 'completed out of', hm_epochs, 'loss:', epoch_loss)
            
        correct = tf.equal(tf.argmax(prediction,1 ), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:', accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))
        
train_neural_network(x)
    


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Epoch 0 completed out of 20 loss: 585853.8907318115
Epoch 1 completed out of 20 loss: 126639.99866867065
Epoch 2 completed out of 20 loss: 75780.45319366455
Epoch 3 completed out of 20 loss: 51372.74943470955
Epoch 4 completed out of 20 loss: 36072.39020574093
Epoch 5 completed out of 20 loss: 25825.30467939377
Epoch 6 completed out of 20 loss: 18712.25850987315
Epoch 7 completed out of 20 loss: 13735.063741649441
Epoch 8 completed out of 20 loss: 10257.865893954175
Epoch 9 completed out of 20 loss: 7403.767608346028
Epoch 10 completed out of 20 loss: 5391.57618905096
Epoch 11 completed out of 20 loss: 4413.210320613318
Epoch 12 completed out of 20 loss: 3397.028035644183
Epoch 13 completed out of 20 loss: 2724.9573218645965
Epoch 14 completed out of 20 loss: 2611.7317064917406
Epoch 15 completed ou

In [39]:

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
import random
import pickle
from collections import Counter

lemmatizer = WordNetLemmatizer()
hm_lines = 10000000

def create_lexicon(pos,neg):
    lexicon = []
    for fi in [pos,neg]:
        with open(fi, 'r') as f:
            contents = f.readlines()
            for l in contents[:hm_lines]:
                all_words = word_tokenize(l.lower())
                lexicon += list(all_words)
                
    lexicon = [lemmatizer.lemmatize(i) for i in lexicon]
    w_counts = Counter(lexicon)
    l2 = []
    for w in w_counts:
        if 1000 > w_counts[w] > 50:
            l2.append(w)
    print(len(l2))
    return l2

def sample_handling(sample, lexicon, classification):
    featureset=[]
    
    with open(sample, 'r') as f:
        contents = f.readlines()
        for l in contents[:hm_lines]:
            current_words = word_tokenize(l.lower())
            current_words = [lemmatizer.lemmatize(i) for i in current_words]
            features = np.zeros(len(lexicon))
            for word in current_words:
                if word.lower() in lexicon:
                    index_value = lexicon.index(word.lower())
                    features[index_value] += 1
                features = list(features)
                featureset.append([features, classification])
                
    return featureset

def create_feature_sets_and_labels(pos,neg,test_size=0.1):
    lexicon = create_lexicon(pos,neg)
    features = []
    features += sample_handling('pos.txt', lexicon,[1,0])
    features += sample_handling('neg.txt', lexicon,[0,1])
    random.shuffle(features)
    
    features = np.array(features)
    testing_size = int(test_size*len(features))
    
    train_x = list(features[:,0][:-testing_size])
    train_y = list(features[:,1][:-testing_size])
    
    test_x = list(features[:,0][-testing_size:])
    test_y = list(features[:,1][-testing_size:])

    return train_x, train_y, test_x, test_y

if __name__ == '__main__':
    train_x,train_y,test_x,test_y = create_feature_sets_and_labels('pos.txt', 'neg.txt')
    with open('sentiment_set.pickle', 'wb') as f:
        pickle.dump([train_x,train_y,test_x,test_y], f)


[nltk_data] Downloading package punkt to /Users/frank/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/frank/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
423
